# Assignment 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 1. Diffusion Limited Aggregation

### A.1 Implement growth model

In [ ]:
# TODO

### A.2 Run a number of growth simulations

In [ ]:
# TODO

### A.3 Investigate the influence of the $\tau$ parameter

In [ ]:
# TODO

### OPTIONAL - Reduce the time required to solve the diffusion equation

In [1]:
# TODO

## 2. Monte Carlo Simulation of DLA

### B.1 Implement the Monte Carlo version of DLA

In [2]:
# TODO

### B.2 Compare the resulting cluster to those obtained with the diffusion equation

In [3]:
# TODO

### C.1 Run the simulation for different values of $p_s$

In [ ]:
# TODO

### C.2 Plot the results

In [ ]:
# TODO

### C.3 How does cluster shape depend on $p_s$?

In [ ]:
# TODO

## 3. The Gray-Scott model

### D.1 Implement the Gray-Scott model in 2D

In [4]:
# TODO

### D.2 Explain the discretization and implementation

In [5]:
# TODO

### D.3 Plot the resulting concentrations of U and V for several parameter choices

In [6]:
# TODO

### D.4 Try adding a small amount of noise

In [7]:
# TODO